FIR　EEG-ECoG 共通 EEG 予測 共通

1. FIR 0, IIR 1, LSTM 2
2. EEG-ECoG間でフィルター共通 0, バラバラ 1
3. 予測対象 EEG 0, ECoG 1
4. 予測の仕方 共通 0, NNをわける（フォルター特性が変わる） 1



In [1]:
data_type = '0111'

In [2]:
# -*- coding: utf_8 -*-  
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn

import copy
import json
import pandas as pd
import numpy as np
import datetime
import math
import time

import chainer
from chainer import cuda, Function, gradient_check, utils, Variable
from chainer import optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from scipy import signal


import pickle
import sys
import random
import h5py

In [3]:
test=h5py.File('Ma/DeepANES.mat')

ECoG = np.array(test['ECoG'])
EEG = np.array(test['EEG'])

In [4]:
EEG = np.delete(EEG, 14, axis=1)

In [5]:
EEG = EEG.astype('float32')
ECoG = ECoG.astype('float32')

In [6]:
# FIR をEEG, ECoG 全部で共通　9808.40625
class TimeSpacePerceptron(Chain):
    def __init__(self):
        self.tt = time_range
        self.eeg_num =16
        self.ecog_num = 1
        super(TimeSpacePerceptron, self).__init__(
            transformECoG = L.Linear(self.eeg_num, self.ecog_num)# EEG to ECoG 空間
            )
        super(TimeSpacePerceptron, self).add_link('IIR_EEG', L.Linear(10, 1))
        super(TimeSpacePerceptron, self).add_link('IIR_ECoG', L.Linear(10, 1))
        #for eeg_ch in range(self.eeg_num):
        #    super(TimeSpacePerceptron, self).add_link('IIR_EEG{0}'.format(eeg_ch), L.Linear(10, 1))
        #for ecog_ch in range(self.ecog_num):
        #    super(TimeSpacePerceptron, self).add_link('IIR_ECoG{0}'.format(ecog_ch), L.Linear(10, 1))
        self.EEG_filtered_prev = [Variable(np.zeros((self.tt, 1)).astype('float32')) for j in range(self.eeg_num)] #前の出力をいれるところ　(RNN)
        self.ECoG_filtered_prev = [Variable(np.zeros((self.tt, 1)).astype('float32')) for j in range(self.ecog_num)] #前の出力をいれるところ
        
    def __call__(self, EEG_t, ECoG_t, train=False):
        EEG_filtered_prev = self.EEG_filtered_prev
        ECoG_filtered_prev = self.ECoG_filtered_prev
        # EEG_t は 16 * time * time_delay
        #EEG FIR
        EEG_filtered = [] #  y0, y1, y2, y3, y4 が入る ch の数だけ配列
        for ch in range(self.eeg_num):
            pre_y = [np.zeros((self.tt, 1)).astype('float32') for  i in range(5)]
            for t in range(5):
                pre_y[t][t:] = EEG_filtered_prev[ch].data[:len(pre_y[t])-t]
                pre_y[t] = Variable(pre_y[t])
            x = F.concat((EEG_t[:, ch, :], pre_y[0], pre_y[1], pre_y[2], pre_y[3], pre_y[4]))
            y = self.__dict__['IIR_EEG'](x)
            EEG_filtered.append(y)
            EEG_filtered_prev[ch] = y
        EEG_filtered = F.concat(tuple(EEG_filtered))
        #ECoG FIR
        ECoG_filtered = [] #  y0, y1, y2, y3, y4 が入る ch の数だけ配列
        for ch in range(self.ecog_num):
            pre_y = [np.zeros((self.tt, 1)).astype('float32') for  i in range(5)]
            for t in range(5):
                pre_y[t][t:] = ECoG_filtered_prev[ch].data[:len(pre_y[t])-t]
                pre_y[t] = Variable(pre_y[t])
            x = F.concat((ECoG_t, pre_y[0], pre_y[1], pre_y[2], pre_y[3], pre_y[4]))
            y = self.__dict__['IIR_ECoG'](x)
            ECoG_filtered.append(y)
            ECoG_filtered_prev[ch] = y
        ECoG_filtered = F.concat(tuple(ECoG_filtered))
        # ここまで FIR
        ecog_space = self.transformECoG(EEG_filtered)
        ecog_space_dropout = F.dropout(ecog_space, train=train, ratio=0.5)
        eeg  = EEG_filtered
        ecog = ECoG_filtered
        self.EEG_filtered_prev =EEG_filtered_prev
        self.ECoG_filtered_prev = ECoG_filtered_prev
        return ecog_space_dropout, ecog



In [7]:
order = 5
time_range = 1999

train_time = [0, time_range]
hamming = signal.hamming(train_time[1]-train_time[0])
#training ぶんだけにする
#EEG_t = np.zeros((EEG.shape[0]-order+1, EEG.shape[1], order))
#ECoG_t = np.zeros((ECoG.shape[0]-order+1, ECoG.shape[1], order))
EEG_t = np.zeros((train_time[1]-train_time[0], EEG.shape[1], order))
ECoG_t = np.zeros((train_time[1]-train_time[0], ECoG.shape[1], order))
EEG_tmp = EEG[train_time[0]:train_time[1], :] *np.array([hamming]*EEG.shape[1]).T
ECoG_tmp = ECoG[train_time[0]:train_time[1], :] *np.array([hamming]*ECoG.shape[1]).T
for t in range(order):
    for ch in range(EEG.shape[1]):
        #EEG_t[:,ch,t] = EEG[:EEG.shape[0]-order+1, ch] 
        EEG_t[t:,ch,t] = EEG_tmp[t:, ch] 
    for ch in range(ECoG.shape[1]):
        #ECoG_t[:,ch,t] = ECoG[:ECoG.shape[0]-order+1, ch] 
        ECoG_t[t:,ch,t] = ECoG_tmp[t:, ch] 
EEG_t = EEG_t[train_time[0]:train_time[1], :, :]
ECoG_t = ECoG_t[train_time[0]:train_time[1], :, :]

In [8]:
saved_data = dict()
for ch_eeg in range(ECoG.shape[1]):
    outout = []
    out_loss_history = []
    saved_data_ch = dict()
    for n in range(1):#20回
        loop = 1 #200
        out = []
        loss_history = np.zeros((loop, 1))
        st = time.time()
        timeSpacePerceptron = TimeSpacePerceptron()
        #optimizer = optimizers.RMSprop()
        optimizer = optimizers.AdaDelta()
        optimizer.setup(timeSpacePerceptron)
        optimizer.add_hook(chainer.optimizer.WeightDecay(0.05))
        #optimizer.add_hook(chainer.optimizer.Lasso(0.5))
        optimizer.add_hook(chainer.optimizer.GradientClipping(0.5))
        #timeSpacePerceptron.zerograds()
        eeg_data = Variable((EEG_t).astype('float32'))
        ecog_data = Variable((ECoG_t[:, ch_eeg,:]).astype('float32'))
        for i in range(loop):
            timeSpacePerceptron.zerograds()
            x1, x2 = timeSpacePerceptron(eeg_data, ecog_data, train=True)
            loss = F.mean_squared_error(x1, x2)
            loss.backward()
            optimizer.prepare()
            optimizer.update()
            loss_history[i] = loss.data
            out.append(copy.deepcopy(timeSpacePerceptron))
        out_loss_history.append(loss_history)
        outout.append(out)
    print(time.time()-st)
    print(loss.data)
    saved_data_ch['loss_history'] = out_loss_history
    saved_data_ch['NN'] = outout
    saved_data[ch_eeg] = saved_data_ch

pickle.dump(saved_data, open('result{0}.pkl'.format(data_type), 'wb'))

0.048422813415527344
13039.4951171875
0.019051074981689453
4831749.5
0.018516063690185547
46472064.0
0.022349119186401367
50066436.0
0.026026010513305664
2237735.0
0.020097970962524414
1750093.875
0.019325971603393555
37876004.0
0.021201133728027344
6402020.0
0.020667076110839844
207577024.0
0.018671035766601562
1416579.625
0.021188974380493164
58473.3359375
0.02140498161315918
101634616.0
0.018592119216918945
171628496.0
0.02130913734436035
324496.40625
0.018724918365478516
27277514.0
0.019116878509521484
585594.375
0.02072596549987793
36471244.0
0.01879596710205078
105074256.0
0.0193331241607666
197531.890625
0.027511119842529297
453858.8125
0.01902294158935547
38740192.0
0.01946711540222168
2996540.5
0.021409988403320312
753427.875
0.019016027450561523
1648346.25
0.01891183853149414
49048772.0
0.03109908103942871
1107172.375
0.019952058792114258
2905006.0
0.025375843048095703
48526432.0
0.02027416229248047
9990132.0
0.022022008895874023
233207.09375
0.026132822036743164
68271888.0
0

In [9]:
#a = pickle.load(open('result{0}.pkl'.format(data_type), 'rb'))